# Importing configurations and modules

In [ ]:
from config.config import *
from callbacks.callbacks import *

In [10]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 4050 Laptop GPU (UUID: GPU-3c98d5c3-b19e-acfa-02d3-5c9d5dfaedd2)
